<h4>Load Beer Data:</h4>

In [103]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/cqa9clje3ye4un611s1nw5fo3')

<h4>Remap Beer Style and Beer Name to integers:</h4>

In [104]:
brewery_name = df['brewery_name'].values
dict_brewery = dict(zip(brewery_name,range(len(brewery_name))))
beer_style = df['beer_style'].values 
dict_style = dict(zip(beer_style,range(len(beer_style))))
beer_name = df['beer_name'].values
dict_name = dict(zip(beer_name,range(len(beer_name))))
reverse_dict_name = {v: k for k, v in dict_name.items()}

In [105]:
df2 = df.copy()
df2['brewery_name'] = df2['brewery_name'].apply(lambda i: dict_brewery[i])
df2['beer_style'] = df2['beer_style'].apply(lambda i: dict_style[i])
df2['beer_name'] = df2['beer_name'].apply(lambda i: dict_name[i])

<h4>Extract Features We Want:</h4>

In [106]:
features = df2[['beer_name','brewery_name','review_overall','review_aroma','review_appearance','beer_style','review_palate','review_taste']]
features = features.dropna(axis=0)
features = features.groupby(['beer_name']).mean()

namez = features.copy()
namez.reset_index(level=0, inplace=True)
print(features)

           brewery_name  review_overall  review_aroma  review_appearance  \
beer_name                                                                  
0          3.000000e+00        1.500000      2.000000           2.500000   
1          3.000000e+00        3.000000      2.500000           3.000000   
2          3.000000e+00        3.000000      2.500000           3.000000   
3          3.000000e+00        3.000000      3.000000           3.500000   
4          9.040000e+02        4.000000      4.500000           4.000000   
14         9.040000e+02        3.833333      3.722222           3.666667   
16         9.040000e+02        2.500000      2.250000           2.500000   
17         9.040000e+02        4.000000      3.000000           4.000000   
62         9.040000e+02        4.166667      4.309524           3.821429   
227        9.040000e+02        3.981595      3.705521           3.849693   
235        9.040000e+02        4.055556      3.666667           3.888889   
272        9

In [115]:
print(features)

           brewery_name  review_overall  review_aroma  review_appearance  \
beer_name                                                                  
0          3.000000e+00        1.500000      2.000000           2.500000   
1          3.000000e+00        3.000000      2.500000           3.000000   
2          3.000000e+00        3.000000      2.500000           3.000000   
3          3.000000e+00        3.000000      3.000000           3.500000   
4          9.040000e+02        4.000000      4.500000           4.000000   
14         9.040000e+02        3.833333      3.722222           3.666667   
16         9.040000e+02        2.500000      2.250000           2.500000   
17         9.040000e+02        4.000000      3.000000           4.000000   
62         9.040000e+02        4.166667      4.309524           3.821429   
227        9.040000e+02        3.981595      3.705521           3.849693   
235        9.040000e+02        4.055556      3.666667           3.888889   
272        9

In [107]:
#Lookup the beer we want then get the row for the features and pass this into the KD Tree
available_beers = namez['beer_name'].apply(lambda i: reverse_dict_name[i]).values
feat_vec = features[['brewery_name', 'review_overall', 'review_aroma',
       'review_appearance', 'beer_style', 'review_palate', 'review_taste']].values

<h4>Create a KDTree for fast nearest neighbors lookup:</h4>

In [108]:
from scipy.spatial import KDTree
tree = KDTree(feat_vec)

<h4>User Selects Beer from List then get the feature corresponding to this beer:</h4>

In [141]:
user_beer = 'Harboe Bear Beer Premium Strong Beer'
query_beer = dict_name[user_beer]
print(query_beer)
print(reverse_dict_name[1582])
query = features.iloc[query_beer].values
print(features.iloc[query_beer])

1582
Harboe Bear Beer Premium Strong Beer
brewery_name         4.595500e+04
review_overall       3.562500e+00
review_aroma         3.281250e+00
review_appearance    3.328125e+00
beer_style           1.509095e+06
review_palate        3.375000e+00
review_taste         3.390625e+00
Name: 45904, dtype: float64


<h4>Query the KDTree with the user selected beer's feature vector:</h4>

In [142]:
distance,index = tree.query(query,k=4)
print(index)

[1582 1550 1546 1548]


<h4>Get User Recommended Beer Name:</h4>

In [143]:
recommended_beers = []
for i in index: 
    beer_id = namez["beer_name"].iloc[i]
    recommended_beers.append(reverse_dict_name[beer_id])

if query_beer in recommended_beers: 
    recommended_beers.remove(query_beer)
print(recommended_beers)

['Harar Beer', "Nessie's Monster Mash", 'Wildcat', 'Marks & Spencer Scottish Ale']


In [150]:
print(namez["beer_name"].iloc[1583])

45955


In [ ]:
#THE REVERSE DICTIONARY IS WRONG WITH THE NAME ID
#FIGURE OUT THE DICTIONARY MAPPING WHY THE QUERY IS NOT MATCHING THE RETURN BEERS